In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
temp=0
for sqrt_value in generator:
    temp=temp+sqrt_value
print (f"the sum of the outputs of the generator for limit = 5 is {temp}")
# the sum of the outputs of the generator for limit = 5 is 8.382332347441762

the sum of the outputs of the generator for limit = 5 is 8.382332347441762


In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [14]:
import dlt
import duckdb
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework')

info = generators_pipeline.run(people_1(),table_name="people1",write_disposition="replace")
info = generators_pipeline.run(people_2(),table_name="people2",write_disposition="replace")

info = generators_pipeline.run(people_1(),table_name="people_append",write_disposition="replace")
info = generators_pipeline.run(people_2(),table_name="people_append",write_disposition="append")

info = generators_pipeline.run(people_1(),table_name="people_merge",write_disposition="replace")
info = generators_pipeline.run(people_2(),table_name="people_merge",write_disposition="merge",primary_key="ID")


# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


Pipeline dlt_ipykernel_launcher load step completed in 0.55 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:///d:\monymovment\zoomcamp_w2\zoomcamp_w2\workshop1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707998784.992083 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people1             │
│ people2             │
│ people_append       │
│ people_merge        │
└─────────────────────┘

In [25]:
def show_table(table_name):
    table = conn.sql(f"SELECT * FROM {table_name}")
    print(f"{table_name}")
    display(table)
    return table

show_table("people1")
show_table("people2")
people_append=show_table("people_append").df()
people_merge=show_table("people_merge").df()

people1


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707998781.1796246 │ saru3U56LQe/Lg │
│     2 │ Person_2 │    27 │ City_A  │ 1707998781.1796246 │ 2Tk90ygl6HUlQQ │
│     3 │ Person_3 │    28 │ City_A  │ 1707998781.1796246 │ FAJWOG6Tvenl4A │
│     4 │ Person_4 │    29 │ City_A  │ 1707998781.1796246 │ wqXI9+n+tWfQ/Q │
│     5 │ Person_5 │    30 │ City_A  │ 1707998781.1796246 │ wCVHyV7CbBGDAw │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘

people2


┌───────┬──────────┬───────┬─────────┬────────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │ occupation │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │  varchar   │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────┼────────────────────┼────────────────┤
│     3 │ Person_3 │    33 │ City_B  │ Job_3      │ 1707998781.9348311 │ +TSh5TqU0q9yuQ │
│     4 │ Person_4 │    34 │ City_B  │ Job_4      │ 1707998781.9348311 │ T3HXhPtVVnpKoA │
│     5 │ Person_5 │    35 │ City_B  │ Job_5      │ 1707998781.9348311 │ bV0XLTMU11tENw │
│     6 │ Person_6 │    36 │ City_B  │ Job_6      │ 1707998781.9348311 │ vC5bQnWBpEs9wg │
│     7 │ Person_7 │    37 │ City_B  │ Job_7      │ 1707998781.9348311 │ J4cNESHb1F305w │
│     8 │ Person_8 │    38 │ City_B  │ Job_8      │ 1707998781.9348311 │ 1qrz2ySGrZI9Cg │
└───────┴──────────┴───────┴─────────┴────────────┴────────────────────┴────────────────┘

people_append


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707998782.6951106 │ Ul92tsIlw9qFkg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707998782.6951106 │ JLO5Nt8GrCEM/g │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707998782.6951106 │ qdun0SDoRvadJg │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707998782.6951106 │ clbW/puS/stKHw │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707998782.6951106 │ d0kk8u4jJPbdsA │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707998783.4818864 │ 3oJzWVqLrhueFw │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707998783.4818864 │ +nCb2yFV7jW++g │ Job_4      │
│     5 │ 

people_merge


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707998784.2304292 │ r9WmKxejRMl7sw │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707998784.2304292 │ gRNfit1kp7emwg │ NULL       │
│     4 │ Person_4 │    34 │ City_B  │ 1707998784.992083  │ r7de+THaXrCirg │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1707998784.992083  │ HchORXVBHXhpYw │ Job_5      │
│     3 │ Person_3 │    33 │ City_B  │ 1707998784.992083  │ IqPR8ddYhvU04A │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1707998784.992083  │ Kam0Yhl44CUUZg │ Job_6      │
│     7 │ Person_7 │    37 │ City_B  │ 1707998784.992083  │ +fvQbXk20zyQPA │ Job_7      │
│     8 │ 

In [24]:
print (f"Sum of Ages {people_append['age'].sum()}")
# Sum of Ages 353
print (f"Sum of Ages {people_merge['age'].sum()}")
# Sum of Ages 266

Sum of Ages 353
Sum of Ages 266
